# sx.affinity.test.ipynb
Notebook for testing.

# Overview



# Input

In [71]:
input_path = "./Inputs/Dev/RenamedPDBBind/"
pdb_name = "1ebw" # f'{pdb_name}.pdb' file in the inputs folder will be used.
pdb_fname = "1ebw_762_protein.pdb"
pdb_fpath = f"{input_path}{pdb_fname}"

nci_fpath = "./Inputs/Dev/dev.nci.csv"


In [72]:
base_pre = f"./Outputs/" # for results

conf_by_chosen = True
# if false and "given_pocket" is 0, only the conformations relating to the given_pocket will be generated.

distinguish_by_timestamp = True 
# if true, the output folder name will be suffiexed with a timestamp 


p2rank = "bash ../p2rank_2.3/prank" # p2rank file path

In [73]:
tankbind_src_folder_path = "../tankbind/"
import sys
sys.path.insert(0, tankbind_src_folder_path)
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import time

In [74]:

timetag = time.strftime("%m%d%H%M")

if distinguish_by_timestamp:
    pre = f"{base_pre}/{pdb_name}-{timetag}"
else:
    pre = f"{base_pre}/{pdb_name}"
    
os.system(f"mkdir -p {pre}")
os.system(f"rm -rf {pre}/sdfs")
os.system(f"mkdir -p {pre}/sdfs")
os.system(f"rm -rf {pre}/PDBs")
os.system(f"mkdir -p {pre}/PDBs")
os.system(f"rm -rf {pre}/p2rank")
os.system(f"mkdir -p {pre}/p2rank")

proteinName = pdb_name
proteinFile = f"{pre}/{proteinName}.pdb"

# get protein feature

In [75]:
from feature_utils import get_protein_feature

In [76]:
from Bio.PDB import PDBParser
from feature_utils import get_clean_res_list
parser = PDBParser(QUIET=True)
protein_dict = {}
proteinName = pdb_name
proteinFile = pdb_fpath
s = parser.get_structure("example", proteinFile)
res_list = list(s.get_residues())
clean_res_list = get_clean_res_list(res_list, ensure_ca_exist=True)
protein_dict[proteinName] = get_protein_feature(clean_res_list)

# p2rank to segment the protein

In [77]:
ds = f"{pre}/protein_list.ds"
with open(ds, "w") as out:
    out.write(f"../../{pdb_fpath}\n")

In [78]:
cmd = f"{p2rank} predict {ds} -o {pre}/p2rank -threads 1"
os.system(cmd)

----------------------------------------------------------------------------------------------
 P2Rank 2.3
----------------------------------------------------------------------------------------------

predicting pockets for proteins from dataset [protein_list.ds]
processing [1ebw_762_protein.pdb] (1/1)
predicting pockets finished in 0 hours 0 minutes 18.147 seconds
results saved to directory [/home/jovyan/TankBind/nciyes/Outputs/1ebw-08020744/p2rank]

----------------------------------------------------------------------------------------------
 finished successfully in 0 hours 0 minutes 20.019 seconds
----------------------------------------------------------------------------------------------


0

In [79]:
import pandas as pd

ligand_fname = "1ebw_762_BEI.mol2"
ligand_fpath = f"{input_path}{ligand_fname}"
from rdkit import Chem
df = pd.DataFrame({"ligand_name":"BEI",
          "protein_name":"1ebw",
        "ligand_fpath":ligand_fpath,
        "smiles":Chem.MolToSmiles(Chem.MolFromMol2File(ligand_fpath))})

In [81]:
ligand_fnames = ["1ebw_762_BEI.mol2", "1ebw_762_BEI_rdkit.mol"]
ligand_fpaths = [f"{input_path}{x}" for x in ligand_fnames]
smiles = []
for _m in ligand_fpaths:
    if "rdkit" in _m:
        smiles.append(Chem.MolToSmiles(Chem.MolFromMolFile(_m)))
    else:
        smiles.append(Chem.MolToSmiles(Chem.MolFromMol2File(_m)))

print(smiles)

from rdkit import Chem
d_df = pd.DataFrame(
    {
        "ligand_name":["BEI", "BEI"],
        "protein_name":["1ebw", "1ebw"],
        "ligand_fpath":ligand_fpaths,
        "smiles":smiles,
        "compound_type": ["Nature", "RDKit"]
        }
    )
d_df

['CN(C(=O)[C@H](OCc1ccccc1)[C@H](O)[C@@H](O)[C@@H](OCc1ccccc1)C(=O)N(C)[C@@H](Cc1ccccc1)C(=O)[O-])[C@@H](Cc1ccccc1)C(=O)[O-]', 'CN(C(=O)[C@H](OCc1ccccc1)[C@H](O)[C@@H](O)[C@@H](OCc1ccccc1)C(=O)N(C)[C@@H](Cc1ccccc1)C(=O)[O-])[C@@H](Cc1ccccc1)C(=O)[O-]']


,ligand_name,protein_name,ligand_fpath,smiles,compound_type
0,BEI,1ebw,./Inputs/Dev/RenamedPDBBind/1ebw_762_BEI.mol2,CN(C(=O)[C@H](OCc1ccccc1)[C@H](O)[C@@H](O)[C@@...,Nature
1,BEI,1ebw,./Inputs/Dev/RenamedPDBBind/1ebw_762_BEI_rdkit...,CN(C(=O)[C@H](OCc1ccccc1)[C@H](O)[C@@H](O)[C@@...,RDKit


In [82]:
info = []
for i, line in tqdm(d_df.iterrows(), total=d_df.shape[0]):
    smiles = line['smiles']
    compound_name = line['ligand_name']
    protein_name = line['protein_name']
    compound_fpath = line['ligand_fpath']
    protein_fname = pdb_fname
    protein_fname = protein_fname.replace(".pdb","")
    compound_type = line["compound_type"]
    print(protein_fname)
    if True:
        p2rankFile = f"{pre}/p2rank/{protein_fname}.pdb_predictions.csv"
        pocket = pd.read_csv(p2rankFile)
        pocket.columns = pocket.columns.str.strip()
        pocket_coms = pocket[['center_x', 'center_y', 'center_z']].values
        for ith_pocket, com in enumerate(pocket_coms):
            com = ",".join([str(a.round(3)) for a in com])
            info.append([protein_name, compound_name, compound_fpath, smiles, compound_type, f"pocket_{ith_pocket+1}", com])
info = pd.DataFrame(info, columns=['protein_name', 'compound_name', 'compound_fpath', 'smiles', "compound_type", 'pocket_name', 'pocket_com'])
info

100%|██████████| 2/2 [00:00<00:00, 72.05it/s]

1ebw_762_protein
1ebw_762_protein


,protein_name,compound_name,compound_fpath,smiles,compound_type,pocket_name,pocket_com
0,1ebw,BEI,./Inputs/Dev/RenamedPDBBind/1ebw_762_BEI.mol2,CN(C(=O)[C@H](OCc1ccccc1)[C@H](O)[C@@H](O)[C@@...,Nature,pocket_1,"12.831,22.867,5.842"
1,1ebw,BEI,./Inputs/Dev/RenamedPDBBind/1ebw_762_BEI.mol2,CN(C(=O)[C@H](OCc1ccccc1)[C@H](O)[C@@H](O)[C@@...,Nature,pocket_2,"16.996,16.021,-19.09"
2,1ebw,BEI,./Inputs/Dev/RenamedPDBBind/1ebw_762_BEI.mol2,CN(C(=O)[C@H](OCc1ccccc1)[C@H](O)[C@@H](O)[C@@...,Nature,pocket_3,"7.603,39.809,11.629"
3,1ebw,BEI,./Inputs/Dev/RenamedPDBBind/1ebw_762_BEI.mol2,CN(C(=O)[C@H](OCc1ccccc1)[C@H](O)[C@@H](O)[C@@...,Nature,pocket_4,"26.531,16.151,-5.906"
4,1ebw,BEI,./Inputs/Dev/RenamedPDBBind/1ebw_762_BEI.mol2,CN(C(=O)[C@H](OCc1ccccc1)[C@H](O)[C@@H](O)[C@@...,Nature,pocket_5,"17.128,40.014,26.253"
5,1ebw,BEI,./Inputs/Dev/RenamedPDBBind/1ebw_762_BEI.mol2,CN(C(=O)[C@H](OCc1ccccc1)[C@H](O)[C@@H](O)[C@@...,Nature,pocket_6,"6.625,34.236,11.795"
6,1ebw,BEI,./Inputs/Dev/RenamedPDBBind/1ebw_762_BEI.mol2,CN(C(=O)[C@H](OCc1ccccc1)[C@H](O)[C@@H](O)[C@@...,Nature,pocket_7,"14.583,33.712,-10.976"
7,1ebw,BEI,./Inputs/Dev/RenamedPDBBind/1ebw_762_BEI_rdkit...,CN(C(=O)[C@H](OCc1ccccc1)[C@H](O)[C@@H](O)[C@@...,RDKit,pocket_1,"12.831,22.867,5.842"
8,1ebw,BEI,./Inputs/Dev/RenamedPDBBind/1ebw_762_BEI_rdkit...,CN(C(=O)[C@H](OCc1ccccc1)[C@H](O)[C@@H](O)[C@@...,RDKit,pocket_2,"16.996,16.021,-19.09"
9,1ebw,BEI,./Inputs/Dev/RenamedPDBBind/1ebw_762_BEI_rdkit...,CN(C(=O)[C@H](OCc1ccccc1)[C@H](O)[C@@H](O)[C@@...,RDKit,pocket_3,"7.603,39.809,11.629"


# construct dataset

In [83]:
import torch
torch.set_num_threads(1)

In [84]:
from torch_geometric.data import Dataset
from utils import construct_data_from_graph_gvp
import rdkit.Chem as Chem    # conda install rdkit -c rdkit if import failure.
from feature_utils import extract_torchdrug_feature_from_mol
from sx_feature_utils import sx_extract_torchdrug_feature_from_mol

In [89]:
class MyDataset_VS(Dataset):
    def __init__(self, root, data=None, protein_dict=None, proteinMode=0, compoundMode=1,
                pocket_radius=20, shake_nodes=None,
                 transform=None, pre_transform=None, pre_filter=None):
        self.data = data
        self.protein_dict = protein_dict
        super().__init__(root, transform, pre_transform, pre_filter)
        print(self.processed_paths)
        self.data = torch.load(self.processed_paths[0])
        self.protein_dict = torch.load(self.processed_paths[1])
        self.proteinMode = proteinMode
        self.pocket_radius = pocket_radius
        self.compoundMode = compoundMode
        self.shake_nodes = shake_nodes
        #self.printflag = True
    @property
    def processed_file_names(self):
        return ['data.pt', 'protein.pt']

    def process(self):
        torch.save(self.data, self.processed_paths[0])
        torch.save(self.protein_dict, self.processed_paths[1])

    def len(self):
        return len(self.data)

    def get(self, idx):
        line = self.data.iloc[idx]
        smiles = line['smiles']
        pocket_com = line['pocket_com']
        pocket_com = np.array(pocket_com.split(",")).astype(float) if type(pocket_com) == str else pocket_com
        pocket_com = pocket_com.reshape((1, 3))
        use_whole_protein = line['use_whole_protein'] if "use_whole_protein" in line.index else False
        compound_type = line['compound_type']
        protein_name = line['protein_name']
        protein_node_xyz, protein_seq, protein_node_s, protein_node_v, protein_edge_index, protein_edge_s, protein_edge_v = self.protein_dict[protein_name]

        compound_fpath = line['compound_fpath']
        
        try:
            if compound_type == "Nature":
                mol = Chem.MolFromMol2File(compound_fpath)
                mol.Compute2DCoords()
            elif compound_type == "RDKit":
                mol = Chem.MolFromMolFile(compound_fpath)
                mol.Compute2DCoords()
                
            coords, compound_node_features, input_atom_edge_list, input_atom_edge_attr_list, pair_dis_distribution = sx_extract_torchdrug_feature_from_mol(mol, has_LAS_mask=True)
        except:
            print("something wrong with ", smiles, "to prevent this stops our screening, we repalce it with a placeholder smiles 'CCC'")
            smiles = 'CCC'
            mol = Chem.MolFromSmiles(smiles)
            mol.Compute2DCoords()
            coords, compound_node_features, input_atom_edge_list, input_atom_edge_attr_list, pair_dis_distribution = extract_torchdrug_feature_from_mol(mol, has_LAS_mask=True)
        # y is distance map, instead of contact map.
        data, input_node_list, keepNode = construct_data_from_graph_gvp(protein_node_xyz, protein_seq, protein_node_s, 
                              protein_node_v, protein_edge_index, protein_edge_s, protein_edge_v,
                              coords, compound_node_features, input_atom_edge_list, input_atom_edge_attr_list,
                              pocket_radius=self.pocket_radius, use_whole_protein=use_whole_protein, includeDisMap=True,
                              use_compound_com_as_pocket=False, chosen_pocket_com=pocket_com, compoundMode=self.compoundMode)
        data.compound_pair = pair_dis_distribution.reshape(-1, 16)
        return data


In [90]:
dataset_path = f"{pre}/dataset/"
os.system(f"rm -r {dataset_path}")
os.system(f"mkdir -p {dataset_path}")
dataset = MyDataset_VS(dataset_path, data=info, protein_dict=protein_dict)

['Outputs/1ebw-08020744/dataset/processed/data.pt', 'Outputs/1ebw-08020744/dataset/processed/protein.pt']


Processing...
Done!


In [91]:
import logging
from torch_geometric.loader import DataLoader
from tqdm import tqdm    # pip install tqdm if fails.
from model import get_model

# take about 2 minutes to screen 10,000 drug candidates.

In [92]:
bias_list = []
batched_energy_list = []

batch_size = 5
device = 'cuda:4' if torch.cuda.is_available() else 'cpu'
# device= 'cpu'
logging.basicConfig(level=logging.INFO)
model = get_model(0, logging, device)
# modelFile = "../saved_models/re_dock.pt"
# self-dock model
modelFile = "../saved_models/self_dock.pt"

model.load_state_dict(torch.load(modelFile, map_location=device))
_ = model.eval()

data_loader = DataLoader(dataset, batch_size=batch_size, follow_batch=['x', 'y', 'compound_pair'], shuffle=False, num_workers=8)
affinity_pred_list = []
y_pred_list = []
for data in tqdm(data_loader):
    data = data.to(device)
    y_pred, affinity_pred = model(data)
    affinity_pred_list.append(affinity_pred.detach().cpu())
    if False:
        # we don't need to save the predicted distance map in HTVS setting.
        for i in range(data.y_batch.max() + 1):
            y_pred_list.append((y_pred[data['y_batch'] == i]).detach().cpu())
affinity_pred_list = torch.cat(affinity_pred_list)

07:49:05   5 stack, readout2, pred dis map add self attention and GVP embed, compound model GIN


100%|██████████| 3/3 [00:03<00:00,  1.04s/it]


In [93]:
info = dataset.data
info['affinity'] = affinity_pred_list

In [94]:
info.to_csv(f"{pre}/TBAff_{pdb_name}_All.csv")

In [100]:
if given_pocket:
    info_given_pocket = info[info["pocket_name"]==given_pocket]
    info_given_pocket.to_csv(f"{pre}/TBAff_{target_name}_{pdb_name}_Given_{given_pocket}.csv")

In [101]:
chosen = info.loc[info.groupby(['protein_name', 'smiles'],sort=False)['affinity'].agg('idxmax')].reset_index()
chosen.to_csv(f"{pre}/TBAff_{target_name}_{pdb_name}_TBChosen.csv")

In [102]:
info.to_csv(f"{pre}/result_info.csv")
if given_pocket:
    info_right_pocket = info[info["pocket_name"]==right_pocket]
    info_right_pocket.to_csv(f"{pre}/result_info_rightpocket.csv")

## Related-Atoms Process

In [103]:
use_related = False
if use_related:
    energy_list = []
    for _batch in batched_energy_list:
        for item in _batch:
            energy_list.append(item)
            
    related_atoms_list = []
    num_atoms_list = []
    for p in info.iterrows():
        sm = p[1]["smiles"]
        #print(sm)
        smiles = get_canonical_smiles(sm)
        #print(smiles)
        mol = Chem.MolFromSmiles(smiles)

        related = []
        for _atom in mol.GetAtoms():
            if _atom.GetAtomicNum() in [7, 8]:
                related.append(_atom.GetIdx())
        related_atoms_list.append(related)
        num_atoms_list.append(mol.GetNumHeavyAtoms())
    
    colsum_energy_list = [i.sum(axis=0) for i in energy_list]
    related_energy_list = [colsum_energy_list[i][related_atoms_list[i]].sum() for i in range(len(colsum_energy_list))]
    total_energy_list = [i.sum() for i in energy_list]

    related_energy_tensor = torch.tensor(related_energy_list)
    total_energy_tensor = torch.tensor(total_energy_list)
    nonrelated_energy_tensor = total_energy_tensor - related_energy_tensor

    bias = bias_list[0]
    outleaky = torch.nn.LeakyReLU()

    related_affinity = outleaky(bias + related_energy_tensor)
    nonrelated_affinity = outleaky(bias + nonrelated_energy_tensor)
    
    info['related_affinity'] = related_affinity
    info['nonrelated_affinity'] = nonrelated_affinity
    
    related_chosen = info.loc[info.groupby(['protein_name', 'smiles'],sort=False)['related_affinity'].agg('idxmax')].reset_index()
    nonrelated_chosen = info.loc[info.groupby(['protein_name', 'smiles'],sort=False)['nonrelated_affinity'].agg('idxmax')].reset_index()
    related_chosen.to_csv(f"{pre}/result_chosen_related.csv")

# 以下暂时不用

In [104]:
from generation_utils import get_LAS_distance_constraint_mask, get_info_pred_distance, write_with_new_coords
# pick one with affinity greater than 7.
chosen = info.loc[info.groupby(['protein_name', 'smiles'],sort=False)['affinity'].agg('idxmax')].reset_index()
chosen = chosen.query("affinity > 7").reset_index(drop=True)
line = chosen.iloc[0]
idx = line['index']
one_data = dataset[idx]
data_with_batch_info = next(iter(DataLoader(dataset[idx:idx+1], batch_size=1, 
                         follow_batch=['x', 'y', 'compound_pair'], shuffle=False, num_workers=1)))
y_pred, affinity_pred = model(data_with_batch_info)

coords = one_data.coords.to(device)
protein_nodes_xyz = one_data.node_xyz.to(device)
n_compound = coords.shape[0]
n_protein = protein_nodes_xyz.shape[0]
y_pred = y_pred.reshape(n_protein, n_compound).to(device).detach()
y = one_data.dis_map.reshape(n_protein, n_compound).to(device)
compound_pair_dis_constraint = torch.cdist(coords, coords)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper__index_select)

In [ ]:
smiles = line['smiles']
print(smiles)
mol = Chem.MolFromSmiles(smiles)
mol.Compute2DCoords()
LAS_distance_constraint_mask = get_LAS_distance_constraint_mask(mol).bool()
info = get_info_pred_distance(coords, y_pred, protein_nodes_xyz, compound_pair_dis_constraint, 
                              LAS_distance_constraint_mask=LAS_distance_constraint_mask,
                              n_repeat=1, show_progress=False)
toFile = f'{base_pre}/one_tankbind.sdf'
new_coords = info.sort_values("loss")['coords'].iloc[0].astype(np.double)
write_with_new_coords(mol, new_coords, toFile)

In [ ]:
import nglview   # conda install nglview -c conda-forge if import failure

proteinName = "6dlo"
proteinFile = f"{base_pre}/{proteinName}.pdb"
view = nglview.show_file(nglview.FileStructure(proteinFile), default=False)
view.add_representation('cartoon', selection='protein', color='white')

predictedFile = f'{base_pre}/one_tankbind.sdf'
rdkit = view.add_component(nglview.FileStructure(predictedFile), default=False)
rdkit.add_ball_and_stick(color='red')
view

In [ ]:
view.render_image()

In [ ]:
view._display_image()